## Grasp reaching, homing motion benchmark
```
BKPIECEkConfigDefault: 	31542.6 ms/50 = 630.9 ms (91.331/3006.085)
KPIECEkConfigDefault: 	27577.7 ms/50 = 551.6 ms (47.669/3007.381)
RRTkConfigDefault: 	48236.9 ms/50 = 964.7 ms (19.026/3013.746)
RRTConnectkConfigDefault: 	22665.6 ms/50 = 453.3 ms (43.77/3006.213)
HOME-BKPIECEkConfigDefault: 	8430.9 ms/50 = 168.6 ms (65.387/438.017)
HOME-KPIECEkConfigDefault: 	2745.9 ms/50 = 54.9 ms (14.473/207.59)
HOME-RRTkConfigDefault: 	3308.4 ms/50 = 66.2 ms (14.098/607.564)
HOME-RRTConnectkConfigDefault: 	2380.3 ms/50 = 47.6 ms (19.585/320.407)
```
- **결과:** 양쪽 모두 RRTConnect가 우수

## set running directory to project source

In [ ]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
import matplotlib.pyplot as plt

## 4.1 PlanningScene

##### initialize CombinedRobot and GeometryScene

In [ ]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)), None)]
                     , connection_list=[False])
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

##### add geometries

In [ ]:
from pkg.geometry.geometry import *

# add environments (fixed=True for non-movable geometries)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.6,-0.3,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0.2,-0.3,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=False)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=True)

obstacle = gscene.create_safe(GEOTYPE.BOX, "obstacle", "base_link", (0.05,0.05,0.05), (0.5,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=False, collision=True)

##### create PlanningScene

In [ ]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder
- Binders (or Actors) are binding points where objects can be attached (or binded)
- Examples are 
  - PlacePlane: plane for object placement
  - Gripper2Tool: 2-finger gripper tool for grasp objects
  - SweepTool: action point to pass waypoints for sweep task

In [ ]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepFramer

In [ ]:
# create PlacePlane on geometry "floor" and "goal"
# when point is not set, the entire upper surface of the geometry becomes valid binding area.
# when point is set, the specific point becomes the only valid binding point.
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

In [ ]:
# add collision boundary for gripper base
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.BOX, "gripper", link_name="indy0_tcp", 
                                dims=(0.02,0.1,0.1), center=(0,0,0.05), rpy=(0,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True)

# add gripper fingers - By setting parent="gripper", the position of geometry can be set relative to the parent geometry
finger1 =  gscene.create_safe(GEOTYPE.BOX, "finger1", link_name="indy0_tcp",
                              dims=(0.04,0.02,0.1), center=(0,0.05,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")
finger2 =  gscene.create_safe(GEOTYPE.BOX, "finger2", link_name="indy0_tcp", 
                              dims=(0.04,0.02,0.1), center=(0,-0.05,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname="grip0", gname="gripper", _type=Gripper2Tool, point=(0,0,0.11), rpy=(-np.pi/2,0,0))

In [ ]:
# Add virtual (no-collision) sweep face. the point is 0.2 m away from the "indy0_tcp" link
# To match the z-direction with the target surface, the geometry is rotated 180 degrees in pitch-axis.
sweep_face =  gscene.create_safe(GEOTYPE.BOX, "sweep_face", link_name="indy0_tcp", 
                                dims=(0.05,0.05,0.001), center=(0,0,0.2), rpy=(0,np.pi,0), 
                                color=(0.2,0.2,0.8,0.1), display=True, fixed=True, collision=False)
                                 
# create SweepTool binder
pscene.create_binder(bname="sweep_face", gname="sweep_face", _type=SweepFramer, point=(0,0,0), rpy=(0,0,0))

##### create_subject
* Subject describes the tasks in the planning scene.
* There are 2 categories in subject:
  1. Object: The object has grip points and placement points for pick&place task
  2. Task: The task is can be any non-physical task. Check SweepLineTask for example
* The subjects can be composed of multiple action points. Examples are:
  1. Grasp2Point: grasping point for 2-finger gripper. 
  2. PlacePoint: The point to place object.
  3. SweepPoint: A waypoint for SweepLineTask.
  * The above 3 action points inherit DirectePoint, for which the orientation is free along z-axis. 
  * If "point" parameter is not set, the entire upper surface is becomes valid action area.

In [ ]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint, SweepFrame
from pkg.planning.constraint.constraint_subject import CustomObject, SweepLineTask

In [ ]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box1, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box1, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box1, [0,0,-0.026], [0,0,0])})

In [ ]:
## create sweep task with 2 waypoints
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepFrame("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepFrame("wp2", wp2, [0,0,0.005], [0,0,0])}
                             )

##### initialize_state
* initialize_state(robot_pose) updates robot pose and gets corresponding binding status of current scene.
* state.node of ('floor', 0) means the first subject (object) is placed on the floor and the second subject (sweep) has passed 0 waypoints

In [ ]:
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

## 4.2 MotionPlanner

##### MoveitPlanner
* *get_available_binding_dict()* gets available binding states for each subject in a dictionary
* *sample_leaf_state samples()* target state with given available_binding_dict and target node
* *rebind_all()* updates binding state and returns the resultant state
* The motions tested in this section are:
  - pick: move the object to "gripper"
  - place: move the object to "goal"
  - sweep: 
    1) approach to waypoint 1
    2) sweep to waypoint 2
    3) return to home pose

In [ ]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)

In [ ]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType, PlannerConfig

In [ ]:
gscene.show_pose(crob.home_pose)

# Comparison test - grasp-reaching motion

In [ ]:
gtimer=GlobalTimer.instance()
gtimer.reset()
# grip 1 - reach the first waypoint
from_state = initial_state.copy(pscene)
from_state.Q = crob.home_pose
to_node = ("gripper", 0)
LOOP_MAX = 50
for i_loop in range(LOOP_MAX):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    available_binding_dict = {'box1': [('handle2', 'grip0', 'gripper')],
                              'sweep': [(None, None, None)]}
    to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    for vv in redundancy_dict.values():
        for v in vv.values():
            v['w']=np.random.uniform(-np.pi/16, np.pi/16)
    with gtimer.block("BKPIECEkConfigDefault", stack=True):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=3, 
                                                                          plannerconfig=PlannerConfig.BKPIECEkConfigDefault)
    with gtimer.block("KPIECEkConfigDefault", stack=True):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=3, 
                                                                          plannerconfig=PlannerConfig.KPIECEkConfigDefault)
    with gtimer.block("RRTkConfigDefault", stack=True):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=3, 
                                                                          plannerconfig=PlannerConfig.RRTkConfigDefault)
    with gtimer.block("RRTConnectkConfigDefault", stack=True):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=3, 
                                                                          plannerconfig=PlannerConfig.RRTConnectkConfigDefault)
    end_state = pscene.rebind_all(binding_list, LastQ)
    home_state = end_state.copy(pscene)
    home_state.Q = crob.home_pose
    with gtimer.block("HOME-BKPIECEkConfigDefault", stack=True):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(end_state, home_state, redundancy_dict, timeout=3, 
                                                                          plannerconfig=PlannerConfig.BKPIECEkConfigDefault)
    with gtimer.block("HOME-KPIECEkConfigDefault", stack=True):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(end_state, home_state, redundancy_dict, timeout=3, 
                                                                          plannerconfig=PlannerConfig.KPIECEkConfigDefault)
    with gtimer.block("HOME-RRTkConfigDefault", stack=True):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(end_state, home_state, redundancy_dict, timeout=3, 
                                                                          plannerconfig=PlannerConfig.RRTkConfigDefault)
    with gtimer.block("HOME-RRTConnectkConfigDefault", stack=True):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(end_state, home_state, redundancy_dict, timeout=3, 
                                                                          plannerconfig=PlannerConfig.RRTConnectkConfigDefault)
    print("=== loop count = {} / {} ===".format(i_loop, LOOP_MAX))
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.01)
    sweep1_state = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
#     raise(RuntimeError("Motion plan failure"))
print(gtimer)

In [ ]:
print(gtimer)

In [ ]:
KEY_LIST = []
for k, v in gtimer.timelist_dict.items():
    v = sorted(v)[:int(len(v)*0.9)]
    plt.plot(np.arange(len(v))+1, sorted(v), 'o')
    KEY_LIST.append(k)
    print("{} - {} ms".format(k, np.mean(v)))
plt.legend(KEY_LIST)
plt.grid()

In [ ]:
KEY_LIST = []
for k, v in gtimer.timelist_dict.items():
    v = sorted(v)[int(len(v)*0.9):]
    plt.plot(np.arange(len(v))+1, sorted(v), 'o')
    KEY_LIST.append(k)
    print("{} - {} ms".format(k, np.mean(v)))
plt.legend(KEY_LIST)
plt.grid()

#### grasp-reaching motion times
* without STOMP
```
BKPIECEkConfigDefault: 	17848.1 ms/50 = 357.0 ms (130.861/1005.99)
RRTkConfigDefault: 	33170.2 ms/50 = 663.4 ms (57.732/1007.893)
RRTConnectkConfigDefault: 	12987.8 ms/50 = 259.8 ms (54.521/1006.136)
```
* with STOMP
```
BKPIECEkConfigDefault: 	43703.2 ms/50 = 874.1 ms (294.829/6632.013)
RRTkConfigDefault: 	50027.4 ms/50 = 1000.5 ms (193.122/6222.588)
RRTConnectkConfigDefault: 	27541.5 ms/50 = 550.8 ms (205.684/3773.618)
```

### visualize trajectory

In [ ]:
gtimer=GlobalTimer.instance()
gtimer.reset()
# grip 1 - reach the first waypoint
from_state = initial_state.copy(pscene)
from_state.Q = crob.home_pose
to_node = ("gripper", 0)
available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
available_binding_dict = {'box1': [('handle2', 'grip0', 'gripper')],
                          'sweep': [(None, None, None)]}
to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
with gtimer.block("RRTConnectkConfigDefault"):
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=3, 
                                                                      plannerconfig=PlannerConfig.RRTConnectkConfigDefault)
gscene.clear_highlight()
gscene.show_motion(Traj, period=0.05)
sweep_face.draw_traj_coords(Traj, "traj_{}".format(sweep_face.name))

In [ ]:
gscene.show_pose(Traj[-2])

In [ ]:
gscene.show_pose((Traj[-1]+Traj[-2])/2)

In [ ]:
gscene.show_pose(Traj[-1])

### check default

In [ ]:
gtimer=GlobalTimer.instance()
gtimer.reset()
from_state = sweep1_state.copy(pscene)
to_node = ("floor", 2)
available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
for _ in range(10):
    gtimer.tic("default")
    Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=10)
    gtimer.toc("default", stack=True)
print(gtimer)

# Compare post-processing

### prepare clearance checker

In [ ]:
from pkg.utils.joint_utils import get_T_dict_foward
from pkg.utils.gjk import get_point_list, set_point_list, get_gjk_distance

def get_dists(gscene, geo_list_1, geo_list_2, Q_dict):
    T_dict = get_T_dict_foward("base_link", gscene.link_names, Q_dict, gscene.urdf_content)

    object_vertice_list = []
    for obj_geo in geo_list_1:
        T = np.matmul(T_dict[obj_geo.link_name], obj_geo.Toff)
        verts, radius = obj_geo.get_vertice_radius()
        verts = np.matmul(verts, T[:3 ,:3].transpose() ) +T[:3 ,3]
        vert_point_list = get_point_list(verts)
        object_vertice_list.append((vert_point_list, radius))

    clearance_vertice_list = []
    for gtem_clr in geo_list_2:
        T = np.matmul(T_dict[gtem_clr.link_name], gtem_clr.Toff)
        verts, radius = gtem_clr.get_vertice_radius()
        verts = np.matmul(verts, T[:3 ,:3].transpose() ) +T[:3 ,3]
        vert_point_list = get_point_list(verts)
        clearance_vertice_list.append((vert_point_list, radius))

    dist_list = []
    for clear_vertice, clear_radius in clearance_vertice_list:
        for object_vertice, object_radius in object_vertice_list:
            dist_list.append(get_gjk_distance(clear_vertice, object_vertice) - clear_radius - object_radius)
    return dist_list

get_dist_tmp = lambda Q: get_dists(gscene, 
                                                   [gtem for gtem in gscene if "indy" in gtem.link_name 
                                                                                           and "indy0_link0" not in gtem.link_name 
                                                                                           and "indy0_link1" not in gtem.link_name 
                                                                                           and gtem.collision], 
                                                   [gtem for gtem in gscene if "indy" not in gtem.link_name and gtem.collision], list2dict(Q, crob.joint_names))

## Prepare Postprocessing states

In [ ]:
gtimer=GlobalTimer.instance()
gtimer.reset(scale=1.0, timeunit='s')
# grip 1 - reach the first waypoint
from_state = initial_state.copy(pscene)
from_state.Q = crob.home_pose
to_node = ("gripper", 0)
to_state_list = []
for _ in range(50):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    available_binding_dict = {'box1': [('handle2', 'grip0', 'gripper')],
                              'sweep': [(None, None, None)]}
    to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    for vv in redundancy_dict.values():
        for v in vv.values():
            v['w']=np.random.uniform(-np.pi/8, np.pi/8)
    for i in range(10):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=3, 
                                                                          plannerconfig=PlannerConfig.RRTConnectkConfigDefault)
        if success:
            break
    to_state = from_state.copy(pscene)
    to_state.Q = LastQ
    to_state_list.append(to_state)

In [ ]:
results_dict = defaultdict(list)
traj_dict = defaultdict(list)
dist_dict = defaultdict(list)

# post-optimizing with CHOMP
***[IMPORTANT]*** Set last item of request_adapters in robots/planning_assets.yaml to chomp/OptimizerAdapter!!

In [ ]:
mplan = MoveitPlanner(pscene)

In [ ]:
to_state = np.random.choice(to_state_list)
Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=10, 
                                                                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, post_opt=True)
gscene.show_motion(Traj, period=0.01)

In [ ]:
gripper.draw_traj_coords(Traj[::5], "traj_{}".format(gripper.name))
print(len(Traj))

In [ ]:
gscene.clear_highlight()

In [ ]:
results_dict = defaultdict(list)
traj_dict = defaultdict(list)
dist_dict = defaultdict(list)

In [ ]:
for to_state in to_state_list:
    with gtimer.block("RRTConnect", stack=True):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=10, 
                                                                          plannerconfig=PlannerConfig.RRTConnectkConfigDefault)
    results_dict["RRTConnect"].append(success)
    if success:
        t_all, Traj = calc_safe_trajectory(2e-2, simplify_traj(Traj), vel_lims=0.5, acc_lims=0.5)
        traj_dict["RRTConnect"].append(sum(np.linalg.norm(Traj[1:]-Traj[:-1], axis=-1)))
        dist_dict["RRTConnect"].append(np.min([np.min(get_dist_tmp(Q)) for Q in Traj]))
    else:
        traj_dict["RRTConnect"].append(0)
        dist_dict["RRTConnect"].append(0)
        
    with gtimer.block("RRTConnect_CHOMP", stack=True):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=10, 
                                                                          plannerconfig=PlannerConfig.RRTConnectkConfigDefault, 
                                                                          post_opt=True)
    results_dict["RRTConnect_CHOMP"].append(success)
    if success:
        t_all, Traj = calc_safe_trajectory(2e-2, simplify_traj(Traj), vel_lims=0.5, acc_lims=0.5)
        traj_dict["RRTConnect_CHOMP"].append(sum(np.linalg.norm(Traj[1:]-Traj[:-1], axis=-1)))
        dist_dict["RRTConnect_CHOMP"].append(np.min([np.min(get_dist_tmp(Q)) for Q in Traj]))
    else:
        traj_dict["RRTConnect_CHOMP"].append(0)
        dist_dict["RRTConnect_CHOMP"].append(0)
    print("-------------------------------")

In [ ]:
print("=== [results] ===")
for k, v in results_dict.items():
    print("{}: {}".format(k, np.round(np.mean(v), 2)))
print("=== [Min/Median/Mean/Max time] ===")
for k, v in gtimer.timelist_dict.items():
    v= np.array(v)[np.where(results_dict[k])[0]]
    print("{}: {} / {} / {} / {}".format(k, 
                                         np.round(np.min(v), 2), np.round(np.median(v), 2), 
                                         np.round(np.mean(v), 2), np.round(np.max(v), 2)))
print("=== [Min/Median/Mean/Max path] ===")
for k, v in traj_dict.items():
    v= np.array(v)[np.where(results_dict[k])[0]]
    print("{}: {} / {} / {} / {}".format(k, 
                                         np.round(np.min(v), 2), np.round(np.median(v), 2), 
                                         np.round(np.mean(v), 2), np.round(np.max(v), 2)))
print("=== [Min/Median/Mean/Max dist] ===")
for k, v in dist_dict.items():
    v= np.array(v)[np.where(results_dict[k])[0]]*1000
    print("{}: {} / {} / {} / {}".format(k, 
                                         np.round(np.min(v), 2), np.round(np.median(v), 2), 
                                         np.round(np.mean(v), 2), np.round(np.max(v), 2)))

In [ ]:
Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=10, 
                                                                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault)
t_all, Traj_safe = calc_safe_trajectory(2e-2, simplify_traj(Traj), vel_lims=0.5, acc_lims=0.5)
gscene.show_motion(Traj_safe, period=0.001)

In [ ]:
import matplotlib.pyplot as plt
dist_mm = np.multiply(1e3, [np.min(get_dist_tmp(Q)) for Q in Traj_safe])
plt.plot(dist_mm)
plt.grid()
print(np.min(dist_mm))

In [ ]:
Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=10, 
                                                                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, post_opt=True)
t_all, Traj_safe_opt = calc_safe_trajectory(2e-2, simplify_traj(Traj), vel_lims=0.5, acc_lims=0.5)
gscene.show_motion(Traj_safe_opt, period=0.001)

In [ ]:
import matplotlib.pyplot as plt
dist_mm = np.multiply(1e3, [np.min(get_dist_tmp(Q)) for Q in Traj_safe_opt])
plt.plot(dist_mm)
plt.grid()
print(np.min(dist_mm))

In [ ]:
# gscene.clear_highlight()
# gripper.draw_traj_coords(Traj, "traj_{}".format(gripper.name))
# print(len(Traj))

# post-optimizing with STOMP
***[IMPORTANT]*** Set last item of request_adapters in robots/planning_assets.yaml to stomp_moveit/StompSmoothingAdapter!!

In [ ]:
mplan = MoveitPlanner(pscene)

In [ ]:
for to_state in to_state_list:
    with gtimer.block("RRTConnect_STOMP", stack=True):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=10, 
                                                                          plannerconfig=PlannerConfig.RRTConnectkConfigDefault, post_opt=True)
    results_dict["RRTConnect_STOMP"].append(success)
    if success:
        traj_dict["RRTConnect_STOMP"].append(sum(np.linalg.norm(Traj[1:]-Traj[:-1], axis=-1)))
        dist_dict["RRTConnect_STOMP"].append(np.min([np.min(get_dist_tmp(Q)) for Q in Traj]))
    else:
        traj_dict["RRTConnect_STOMP"].append(0)
        dist_dict["RRTConnect_STOMP"].append(0)
    print("-------------------------------")

In [ ]:
Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=10, 
                                                                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, post_opt=True)
gscene.show_motion(Traj, period=0.01)

# optimal path planning with RRT*

In [ ]:
from_state = initial_state.copy(pscene)
from_state.Q = crob.home_pose
to_node = ("gripper", 0)
for to_state in to_state_list:
    with gtimer.block("RRTstar", stack=True):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=30, 
                                                                          plannerconfig=PlannerConfig.RRTstarkConfigDefault)
    results_dict["RRTstar"].append(success)
    if success:
        traj_dict["RRTstar"].append(sum(np.linalg.norm(Traj[1:]-Traj[:-1], axis=-1)))
        dist_dict["RRTstar"].append(np.min([np.min(get_dist_tmp(Q)) for Q in Traj]))
    else:
        traj_dict["RRTstar"].append(0)
        dist_dict["RRTstar"].append(0)
    print("-------------------------------")

In [ ]:
Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=10, 
                                                                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, post_opt=True)
gscene.show_motion(Traj, period=0.01)

In [ ]:
print("=== [results] ===")
for k, v in results_dict.items():
    print("{}: {}".format(k, np.round(np.mean(v), 2)))
print("=== [Min/Median/Mean/Max time] ===")
for k, v in gtimer.timelist_dict.items():
    v= np.array(v)[np.where(results_dict[k])[0]]
    print("{}: {} / {} / {} / {}".format(k, 
                                         np.round(np.min(v), 2), np.round(np.median(v), 2), 
                                         np.round(np.mean(v), 2), np.round(np.max(v), 2)))
print("=== [Min/Median/Mean/Max path] ===")
for k, v in traj_dict.items():
    v= np.array(v)[np.where(results_dict[k])[0]]
    print("{}: {} / {} / {} / {}".format(k, 
                                         np.round(np.min(v), 2), np.round(np.median(v), 2), 
                                         np.round(np.mean(v), 2), np.round(np.max(v), 2)))
print("=== [Min/Median/Mean/Max dist] ===")
for k, v in dist_dict.items():
    v= np.array(v)[np.where(results_dict[k])[0]]*1000
    print("{}: {} / {} / {} / {}".format(k, 
                                         np.round(np.min(v), 2), np.round(np.median(v), 2), 
                                         np.round(np.mean(v), 2), np.round(np.max(v), 2)))

### for record, time step comparison

In [ ]:
print("TIMESTEP 100")
print("=== [Mean time] ===")
print(gtimer)
print("=== [Median time] ===")
for k, v in gtimer.timelist_dict.items():
    print("{}: {}".format(k, np.round(np.median(v), 2)))
print("=== [results] ===")
for k, v in results_dict.items():
    print("{}: {}".format(k, np.round(np.mean(v), 2)))

In [ ]:
print("STOMP TIMESTEP 5")
print("=== [Mean time] ===")
print(gtimer)
print("=== [Median time] ===")
for k, v in gtimer.timelist_dict.items():
    print("{}: {}".format(k, np.round(np.median(v), 2)))
print("=== [results] ===")
for k, v in results_dict.items():
    print("{}: {}".format(k, np.round(np.mean(v), 2)))

# Test TrajOpt - Fail: bad_function_call

In [ ]:
gtimer=GlobalTimer.instance()
gtimer.reset(scale=1.0, timeunit='s')
# grip 1 - reach the first waypoint
from_state = initial_state.copy(pscene)
from_state.Q = crob.home_pose
to_node = ("gripper", 0)
results_dict = defaultdict(list)
for _ in range(1):
    available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
    available_binding_dict = {'box1': [('handle2', 'grip0', 'gripper')],
                              'sweep': [(None, None, None)]}
    to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
    for vv in redundancy_dict.values():
        for v in vv.values():
            v['w']=np.random.uniform(-np.pi/16, np.pi/16)
    for i in range(10):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=3, 
                                                                          plannerconfig=PlannerConfig.RRTConnectkConfigDefault)
        if success:
            break
    with gtimer.block("TrajOptDefault"):
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=30, 
                                                                          plannerconfig=PlannerConfig.TrajOptDefault)
        results_dict["TrajOptDefault"].append(success)
    print("-------------------------------")
if success:
    pscene.set_object_state(from_state)
    gscene.show_motion(Traj, period=0.01)
    sweep1_state = pscene.rebind_all(binding_list, LastQ)
else:
    print("Solution not found. Please try again, try to find error if it keeps failing.")
#     raise(RuntimeError("Motion plan failure"))
print("")
print("============= [Timer] =============")
print(gtimer)
print("============= [Result] =============")
for k, v in results_dict.items():
    print("{}: {}".format(k, np.round(np.mean(v), 2)))

### test time

In [ ]:
import random
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
cs_types = [ConstrainedSpaceType.PROJECTED, ConstrainedSpaceType.ATLAS, ConstrainedSpaceType.TANGENTBUNDLE]

In [ ]:
gtimer = GlobalTimer.instance()
gtimer.reset()
# sweep motion 2 - sweep "floor" to the second waypoint. Constrained motion planning takes longer time
suc_dict = defaultdict(list)
max_dict = defaultdict(list)
std_dict = defaultdict(list)
time_dict = defaultdict(list)
from_state = sweep1_state.copy(pscene)
to_node = ("floor", 2)
available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)

In [ ]:
for _ in range(10):    
    random.shuffle(cs_types)
    for cs_type in cs_types:
        gtimer.tic(cs_type.name)
        Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, redundancy_dict, timeout=10, cs_type=cs_type)
        elapsed = gtimer.toc(cs_type.name)
        suc_dict[cs_type.name].append(success)
        time_dict[cs_type.name].append(elapsed)
        if success:
            T_ref=floor.get_tf(crob.home_dict)
            Z_list = []
            for q in Traj:
                T_tool=sweep_face.get_tf(list2dict(q, crob.joint_names))
                Z_list.append(np.matmul(SE3_inv(T_ref), T_tool)[2,3])
            std_dict[cs_type.name].append(np.std(Z_list))
            max_dict[cs_type.name].append(np.max(Z_list)-np.min(Z_list))
    print(gtimer)

In [ ]:
import matplotlib.pyplot as plt
keys = ["PROJECTED", "ATLAS", "TANGENTBUNDLE"]

In [ ]:
plt.figure(figsize=(15,6))
for i_k, k in enumerate(keys):
    v = time_dict[k]
    plt.subplot(1,3,i_k+1)
    plt.plot(v, '.')
    print("time_dict {}: {}".format(k, np.mean(v)))
    print("time median {}: {}".format(k, np.median(v)))

In [ ]:
plt.figure(figsize=(15,6))
for i_k, k in enumerate(keys):
    v = suc_dict[k]
    plt.subplot(1,3,i_k+1)
    plt.plot(v, '.')
    print("suc_dict {}: {}".format(k, np.mean(v)))

In [ ]:
plt.figure(figsize=(15,6))
for i_k, k in enumerate(keys):
    v = std_dict[k]
    plt.subplot(1,3,i_k+1)
    plt.plot(v, '.')
    print("std_dict {}: {}".format(k, np.mean(v)))

In [ ]:
plt.figure(figsize=(15,6))
for i_k, k in enumerate(keys):
    v = max_dict[k]
    plt.subplot(1,3,i_k+1)
    plt.plot(v, '.')
    print("max_dict {}: {}".format(k, np.mean(v)))